# CS/ECE/ISyE 524 - Spr 2021 - HW 9
### Jingde Wan

# 1. Checked luggage.

You're trying to pack as many souvenirs as possible to bring home from your trip, but your suitcase has a limited capacity. It can hold a maximum of 30 pounds of weight and 15 gallons of volume. Which souvenirs should you pack? The weights and volumes are as follows:

|Souvenir number |1 |2 |3 |4 |5 |6 |7 |8 |9 |10|
|:---------------|--|--|--|--|--|--|--|--|--|--|
|Weight (lbs)    |5 |6 |7 |6 |4 |6 |7 |3 |8 |5 |
|Volume (gal)    |2 |4 |5 |3 |3 |2 |3 |1 |2 |4 |

In [2]:
using Pkg
Pkg.add("Cbc")

 Installing known registries into `C:\Users\jingd\.julia`
      Added registry `General` to `C:\Users\jingd\.julia\registries\General`
  Resolving package versions...
  Installed Cgl_jll ─ v0.60.2+6
  Installed Cbc_jll ─ v2.10.5+2
  Installed Cbc ───── v0.8.0
Updating `C:\Users\jingd\.julia\environments\v1.5\Project.toml`
  [9961bab8] + Cbc v0.8.0
Updating `C:\Users\jingd\.julia\environments\v1.5\Manifest.toml`
  [9961bab8] + Cbc v0.8.0
  [38041ee0] + Cbc_jll v2.10.5+2
  [3830e938] + Cgl_jll v0.60.2+6
   Building Cbc → `C:\Users\jingd\.julia\packages\Cbc\JdG60\deps\build.log`


In [3]:
Pkg.status()

Status `C:\Users\jingd\.julia\environments\v1.5\Project.toml`
  [336ed68f] CSV v0.8.3
  [9961bab8] Cbc v0.8.0
  [e2554f3b] Clp v0.8.3
  [a93c6f00] DataFrames v0.22.5
  [e2685f51] ECOS v0.12.2
  [60bf3e95] GLPK v0.14.6
  [2e9cd046] Gurobi v0.8.0
  [7073ff75] IJulia v1.23.1
  [c601a237] Interact v0.10.3
  [b6b21f68] Ipopt v0.6.5
  [4076af6c] JuMP v0.21.6
  [86f7a689] NamedArrays v0.9.4
  [91a5bcdd] Plots v1.10.6
  [d330b81b] PyPlot v2.9.0
  [c946c3f1] SCS v0.7.1


In [4]:
# problem data (we'll use an array here)
weights = [ 5, 6, 7, 6, 4, 6, 7, 3, 8, 5 ]
volumes = [ 2, 4, 5, 3, 3, 2, 3, 1, 2, 4 ]
n = length(volumes) # total number of souvenirs

# insert your code here
using JuMP, Cbc

m = Model(Cbc.Optimizer)

@variable(m, x[1:10], Bin)

@constraint(m, sum(weights[i]*x[i] for i in 1:n) <= 30)
@constraint(m, sum(volumes[i]*x[i] for i in 1:n) <= 15)

@objective(m, Max, sum(x[i] for i in 1:n))

optimize!(m);
println(" binary solution vector is ",value.(x))
println(objective_value(m))

 binary solution vector is [1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0]
6.0
Welcome to the CBC MILP Solver 
Version: 2.10.5 
Build Date: Mar 11 2021 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 6.09091 - 0.00 seconds
Cgl0004I processed model has 2 rows, 10 columns (10 integer (10 of which binary)) and 20 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.272727
Cbc0038I Solution found of -6
Cbc0038I Before mini branch and bound, 8 integers at bound fixed and 0 continuous
Cbc0038I Mini branch and bound did not improve solution (0.00 seconds)
Cbc0038I After 0.00 seconds - Feasibility pump exiting with objective of -6 - took 0.00 seconds
Cbc0012I Integer solution of -6 found by feasibility pump after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective -6, took 1 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 8 variables fix

#### We should pick souvenir 1,4,5,6,8,10

# 2. Comquat Computers

Comquat owns four production plants at which personal computers are produced. Comquat can sell up to 20,000 computers per year at a price of \$3,500 per computer. For each plant the production capacity, cost per computer, and fixed cost of operating the plant for a year are given below. Determine how Comquat can maximize its yearly profit from computer production.

| Plant | Production capacity | Plant fixed cost (\$ Million) | Cost per computer (\$) |
|:-----:|:-------------------:|:-----------------------------:|:----------------------:|
| 1     | 10,000              |   9                           |  1,000                 |
| 2     |  8,000              |   5                           |  1,700                 |
| 3     |  9,000              |   3                           |  2,300                 |
| 4     |  6,000              |   1                           |  2,900                 |

In [13]:
using JuMP, Cbc
m = Model(Cbc.Optimizer)
# set_optimizer_attribute(m, "OutputFlag", 0)

price     = 3.5e3                       # price at which we can sell a single computer 
                                        # (regardless of where it is produced)
cap       = 20000                       # maximum computers that can be sold per year

capacity  = [10000, 8000, 9000, 6000]   # production capacity for each plant
fixedcost = [9e6, 5e6, 3e6, 1e6]        # fixed cost for each plant
itemcost  = [1e3, 1.7e3, 2.3e3, 2.9e3]  # cost per computer for each factory


# insert your code here, with integer variable vector z and real variable vector x
n = length(capacity)

@variable(m, x[1:n]>=0)# may be wrong!!!
@variable(m, z[1:n], Bin)

# capacity
for i in 1:n
    @constraint(m, x[i] <= capacity[i])  
end

# up to 20,000 computers
@constraint(m, sum(x[i] for i in 1:n) <= cap)

# if x>0 then z=1
@constraint(m, x .<= 20000*z)

@objective(m, Max, sum(3500*x[i]-itemcost[i]*x[i]-z[i]*fixedcost[i] for i in 1:n))

optimize!(m)
println("The maximal profit is: ", objective_value(m))
println("binary vector for factory number 1 to 4: ", value.(z))
println("production for factory number 1 to 4:", value.(x))
println("Here is a tally of factory number, whether it's open or not, and how many computers it should produce.")
println(value.([1:4 z x]))

The maximal profit is: 2.56e7
binary vector for factory number 1 to 4: [1.0, 1.0, 0.0, 1.0]
production for factory number 1 to 4:[10000.0, 8000.0, 0.0, 2000.0]
Here is a tally of factory number, whether it's open or not, and how many computers it should produce.
[1.0 1.0 10000.0; 2.0 1.0 8000.0; 3.0 0.0 0.0; 4.0 1.0 2000.0]
Welcome to the CBC MILP Solver 
Version: 2.10.5 
Build Date: Mar 11 2021 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 3.5e+07 - 0.00 seconds
Cgl0004I processed model has 5 rows, 8 columns (4 integer (4 of which binary)) and 12 elements
Cbc0038I Initial state - 1 integers unsatisfied sum - 0.222222
Cbc0038I Pass   1: suminf.    0.00000 (0) obj. -2.54e+07 iterations 3
Cbc0038I Solution found of -2.54e+07
Cbc0038I Relaxing continuous gives -2.54e+07
Cbc0038I Before mini branch and bound, 3 integers at bound fixed and 3 continuous
Cbc0038I Full problem 5 rows 8 columns, reduced to 0 rows 0 columns
Cbc0038I Mini branch 

#### Comquat should make plant 1,2,4 on production with amount 10000, 8000, 2000 computers.

# 3. ABC Investments

ABC Inc. is considering several investment options.  Each option has a minimum investment required as well as a maximum investment allowed. These restrictions, along with the expected return are summarized in the following table (figures are in millions of dollars):

|Option | Minimum investment | Maximum investment | Expected return (%)  |
|:-----:|:------------------:|:------------------:|:--------------------:|
|  1    |      3             |      27            |     13               |
|  2    |      2             |      12            |      9               |
|  3    |      9             |      35            |     17               |
|  4    |      5             |      15            |     10               |
|  5    |     12             |      46            |     22               |
|  6    |      4             |      18            |     12               |

Because of the high-risk nature of Option 5, company policy requires that the total amount invested in Option 5 be no more that the combined amount invested in Options 2, 4 and 6.  In addition, if an investment is made in Option 3, it is required that at least a minimum investment be made in Option 6.  ABC has \$80 million to invest and obviously wants to maximize its total expected return on investment.  Which options should ABC invest in, and how much should be invested?

In [19]:
using JuMP, Cbc
m = Model(Cbc.Optimizer)
# set_optimizer_attribute(m, "OutputFlag", 0)

lows  = [  3,  2,  9,  5, 12,  4 ]  # minimum investment
highs = [ 27, 12, 35, 15, 46, 18 ]  # maximum investment
ret   = [ 13,  9, 17, 10, 22, 12 ]  # expected return

# your code here

n = length(highs)
@variable(m, x[1:n]>=0)
@variable(m, z[1:n], Bin)

for i in 1:n
    @constraint(m, x[i] >= lows[i]*z[i])
    @constraint(m, x[i] <= highs[i])
end
    
@constraint(m, x[5] <= x[2]+x[4]+x[6])
@constraint(m, z[3] <= z[6])
@constraint(m, x .<= 80*z)
@constraint(m, sum(x[i] for i in 1:n) <= 80)

@objective(m, Max, sum(0.01*x[i]*ret[i] for i in 1:n))

optimize!(m)
println("The maximum return on investment (net profit) is: \$", objective_value(m), " million.")
println("This is an average return of: ", 100*objective_value(m)/value(sum(x)), " %")
println("binary vector for option number 1 to 6: ", value.(z))
println("amount of investment for option number 1 to 6:", value.(x))
println("Here is a tally of each investment, whether we invest or not, and how much we invest (in millions of \$).")
print(value.([1:6 z x]))

The maximum return on investment (net profit) is: $13.500000000000004 million.
This is an average return of: 16.875000000000007 %
binary vector for option number 1 to 6: [0.0, 0.0, 1.0, 1.0, 1.0, 1.0]
amount of investment for option number 1 to 6:[0.0, 0.0, 35.0, 5.000000000000001, 22.499999999999996, 17.5]
Here is a tally of each investment, whether we invest or not, and how much we invest (in millions of $).
[1.0 0.0 0.0; 2.0 0.0 0.0; 3.0 1.0 35.0; 4.0 1.0 5.000000000000001; 5.0 1.0 22.499999999999996; 6.0 1.0 17.5]Welcome to the CBC MILP Solver 
Version: 2.10.5 
Build Date: Mar 11 2021 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 13.51 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 15 rows, 12 columns (6 integer (6 of which binary)) and 36 elements
Cbc0038I Initial state - 2 integers unsatisfied sum - 0.8
Cbc0038I Pass   1: suminf.    0.00000 (0) obj. -8.11 ite

#### We should invest in option 3, 4, 5, 6 with amount of 35, 5, 22.5, 17.5 billion dollors respectively.

# 4. Paint production

As part of its weekly production, a paint company produces five batches of paints, always the same, for some big clients who have a stable demand.  Every paint batch is produced in a single production process, all in the same blender that needs to be cleaned between each batch.  The durations of blending paint batches 1 to 5 are 40, 35, 45, 32 and 50 minutes respectively.  The cleaning times depend of the colors and the paint types.  For example, a long cleaning period is required if an oil-based paint is produced after a water-based paint, or to produce white paint after a dark color.  The times are given in minutes in the following matrix $A$ where $A_{ij}$ denotes the cleaning time after batch $i$ if it is followed by batch $j$.

$$
A = \begin{bmatrix}
 0&11& 7&13&11 \\
 5& 0&13&15&15 \\
13&15& 0&23&11 \\
 9&13& 5& 0& 3 \\
 3& 7& 7& 7& 0
\end{bmatrix}
$$

Since the company has other activities, it wishes to deal with this weekly production in the shortest possible time (blending and cleaning).  What is the corresponding order of paint batches?  The order will be applied every week, so the cleaning time between the last batch of one week and the first of the following week needs to be accounted for in the total duration of cleaning.
                

In [37]:
# This is a traveling salesman problem! cities correspond to paint batches, and the time it takes to travel
# from one city to the next is the time it takes to clean the blender in between two particular batches.

# Because the cleaning time between the last batch of one week and the first batch of the following week needs
# to be accounted for, this is EXACTLY a traveling salesman problem!

# A[i,j] is the time it takes to travel j --> i.
A = [  0  11   7  13  11
       5   0  13  15  15
      13  15   0  23  11
       9  13   5   0   3
       3   7   7   7   0 ]

# time it takes to blend a particular batch. This information isn't important, because we need to blend each batch
# every week no matter what order they're blended in. So this is a fixed cost!
b = [ 40, 35, 45, 32, 50 ]

using JuMP, Cbc
m = Model(Cbc.Optimizer)
# set_optimizer_attribute(m, "OutputFlag", 0)

n = length(b)  # number of paints

# your code here
N = size(A,1)
@variable(m, x[1:n,1:n], Bin)


for i in 1:n
    @constraint(m, sum(x[i,j] for j in 1:n) == 1)
end

for j in 1:n
    @constraint(m, sum(x[i,j] for i in 1:n) == 1)
end

for i in 1:n
    @constraint(m, x[i,i] == 0)
end

@objective(m, Min, sum(x[i,j]*A[i,j] for i in 1:n, j in 1:n))

@variable(m, u[1:5])
for i in 1:5
    for j in 2:5
        @constraint(m, u[i]-u[j]+N*x[i,j] <= N-1)
    end
end


optimize!(m)
print(X)
println("\nOptimal ordering is:")
X = value.(x);
# println(X);
for i = 1:n
    for j=1:n
        if (X[i,j] >= .99)
            println(j)
        end
    end
end

println("and the minimum cleanup time is: ", objective_value(m), " min, (plus ", sum(b), " min of blending)")

[0.0 0.0 0.0 1.0 0.0; 1.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 1.0; 0.0 0.0 1.0 0.0 0.0; 0.0 1.0 0.0 0.0 0.0]
Optimal ordering is:
4
1
5
3
2
and the minimum cleanup time is: 41.0 min, (plus 202 min of blending)
Welcome to the CBC MILP Solver 
Version: 2.10.5 
Build Date: Mar 11 2021 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 37.8 - 0.00 seconds
Cgl0002I 5 variables fixed
Cgl0004I processed model has 22 rows, 24 columns (20 integer (20 of which binary)) and 76 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 8 integers unsatisfied sum - 1.6
Cbc0038I Pass   1: suminf.    1.60000 (4) obj. 39.4 iterations 5
Cbc0038I Solution found of 43
Cbc0038I Relaxing continuous gives 43
Cbc0038I Before mini branch and bound, 9 integers at bound fixed and 0 continuous
Cbc0038I Full problem 22 rows 24 columns, reduced to 13 rows 7 columns
Cbc0038I Mini branch and bound improved solution from 43 to 41 (0.01 seconds)
Cbc0038

#### The optimal order is 4-1-5-3-2 and the shortest time is 41+202=243